<a href="https://colab.research.google.com/github/harsha4240/harsha/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies (this is mostly for Google Colab, as the other dependences are available by default in Colab)
try:
  import datasets, evaluate, accelerate
  import gradio as gr
except ModuleNotFoundError:
  !pip install -U datasets evaluate accelerate gradio # -U stands for "upgrade" so we'll get the latest version by default
  import datasets, evaluate, accelerate
  import gradio as gr

import random

import numpy as np
import pandas as pd

import torch
import transformers

print(f"Using transformers version: {transformers.__version__}")
print(f"Using datasets version: {datasets.__version__}")
print(f"Using torch version: {torch.__version__}")

Using transformers version: 4.48.2
Using datasets version: 3.2.0
Using torch version: 2.5.1+cu124


In [2]:
from datasets import load_dataset

dataset = load_dataset("mrdbourke/learn_hf_food_not_food_image_captions")

dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/250 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 250
    })
})

In [3]:
dataset.column_names

{'train': ['text', 'label']}

In [6]:
dataset["train"][0]

{'text': 'Creamy cauliflower curry with garlic naan, featuring tender cauliflower in a rich sauce with cream and spices, served with garlic naan bread.',
 'label': 'food'}

In [7]:
import random

random_indexs = random.sample(range(len(dataset["train"])), 5)
random_samples = dataset["train"][random_indexs]

print(f"[INFO] Random samples from dataset:\n")
for item in zip(random_samples["text"], random_samples["label"]):
  print(f"text: {item[0]} | Label: {item[1]}")

[INFO] Random samples from dataset:

text: A boy giving his dog a bath in the backyard | Label: not_food
text: Pizza with a white sauce base, topped with spinach and artichokes | Label: food
text: A pair of slices from a barbecue chicken pizza | Label: food
text: Three apples and two oranges next to each other on a kitchen table | Label: food
text: Spicy prawn curry with fresh mint garnish, featuring juicy prawns in a fiery sauce with onions and tomatoes, finished with mint leaves. | Label: food


In [8]:
dataset["train"].unique("label")

['food', 'not_food']

In [9]:
from collections import Counter

Counter(dataset["train"]["label"])

Counter({'food': 125, 'not_food': 125})

In [15]:
food_not_food_df = pd.DataFrame(dataset["train"])
food_not_food_df.sample(7)

,text,label
47,Wooden hanger holding clothes on a rack,not_food
171,Red ceramic mug sitting next to a coffee maker,not_food
141,A slice of pizza with a spicy buffalo chicken ...,food
1,Set of books stacked on a desk,not_food
186,Traditional Japanese flavored sushi roll with ...,food
245,Standing floor lamp providing light next to an...,not_food
236,"Pizza with a dessert twist, featuring a sweet ...",food
